In [1]:
%matplotlib inline

import time
import os
import pprint
import sys
import IPython
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
sys.path.append('../')
from model.DataProvider import DataProvider
from model.EarlyStopping import EarlyStopping
from model.NetSaver import NetSaver
from model.MRUNet import MRUNet
from model import Loss
from model import Trainer
from model import FeatureExtraction
from model import DataArgument
from model.STFT_Module import STFT_Module
from lib import AudioModule
from visualizer import visualize_loss
from visualizer import visualize_spec


class Train():
        def __init__(self, epoch_num=2000, batch_size=20, lr_init = 0.0001, fs = 16000, sec = 6, train_data_num=2000, valid_data_num=200, sample_len=66304):
                self.epoch_num= epoch_num
                self.batch_size = batch_size
                self.input_shape = None
                self.lr_init = lr_init
                self.train_iter = None
                self.valid_iter = None
                self.train_loss_list = []
                self.valid_loss_list = []
                self.fs = fs
                self.sec = sec
                self.stft_params = {
                        "frame_length": 1024,
                        "frame_step":  256,
                        "fft_length": 1024,
                }
                
                self.mr_stft_params = {
                    "frame_length": 512,
                    "frame_step": 128,
                    "fft_length": 512
                }
                self.epsilon = 1e-4
            
        def __model(self, tf_mix, tf_target, tf_lr):
                 # define model flow
                # stft
                stft_module = STFT_Module(
                        frame_length = self.stft_params["frame_length"], 
                        frame_step= self.stft_params["frame_step"], 
                        fft_length = self.stft_params["fft_length"],
                        epsilon = self.epsilon,
                        pad_end=False
                )
                
                mr_stft_module = STFT_Module(
                        frame_length = self.mr_stft_params["frame_length"], 
                        frame_step= self.mr_stft_params["frame_step"], 
                        fft_length = self.mr_stft_params["fft_length"],
                        epsilon = self.epsilon,
                        pad_end=False
                )
                
                
                # mix data transform
                tf_spec_mix = stft_module.STFT(tf_mix)
                print(tf_mix.shape)
                print(tf_spec_mix.shape)
                tf_amp_spec_mix = stft_module.to_amp_spec(tf_spec_mix, normalize =False)
                tf_mag_spec_mix = tf.log(tf_amp_spec_mix + self.epsilon)
                tf_mag_spec_mix = tf.expand_dims(tf_mag_spec_mix, -1)# (Batch, Time, Freq, Channel))
                tf_amp_spec_mix = tf.expand_dims(tf_amp_spec_mix, -1)
                tf_f_512_mag_spec_mix = stft_module.to_F_512(tf_mag_spec_mix)
                
                #mr mix data transform
                tf_mr_spec_mix = mr_stft_module.STFT(tf_mix)
                tf_mr_spec_mix = tf_mr_spec_mix[:, 1:513, :]
                tf_mr_amp_spec_mix = stft_module.to_amp_spec(tf_mr_spec_mix, normalize =False)
                tf_mr_mag_spec_mix = tf.log(tf_mr_amp_spec_mix + self.epsilon)
                tf_mr_mag_spec_mix = tf.expand_dims(tf_mr_mag_spec_mix, -1)# (Batch, Time, Freq, Channel))
                tf_mr_f_256_mag_spec_mix = tf_mr_mag_spec_mix[:, :, :256]
                
                # target data transform
                tf_spec_target = stft_module.STFT(tf_target)
                tf_spec_target = stft_module.to_T_256(tf_spec_target) # cut time dimensiton to 256 for u-net architecture
                
                tf_amp_spec_target = stft_module.to_amp_spec(tf_spec_target, normalize=False)
                tf_amp_spec_target = tf.expand_dims(tf_amp_spec_target, -1)
                 
                mr_u_net = MRUNet(
                        input_shape = (
                                tf_f_512_mag_spec_mix.shape[1:]
                        ),
                        mr_input_shape = (
                               tf_mr_f_256_mag_spec_mix.shape[1:]
                        )
                )
            
                tf_est_masks = mr_u_net(tf_f_512_mag_spec_mix, tf_mr_f_256_mag_spec_mix)
                
                #F: 512  → 513
                zero_pad = tf.zeros_like(tf_mag_spec_mix)
                zero_pad = tf.expand_dims(zero_pad[:,:,1,:], -1)
                tf_est_masks = tf.concat( [tf_est_masks, zero_pad], 2)
                
                tf_ora_masks = Masks.iaf(tf_amp_spec_mix, tf_amp_spec_target,self.epsilon)
                tf_loss = 10 * Loss.mean_square_error(tf_est_masks, tf_ora_masks)
                tf_train_step = Trainer.Adam(tf_loss, tf_lr)
                
                return tf_train_step, tf_loss, tf_est_spec, tf_amp_spec_target, tf_mag_spec_mix,   tf_spec_mix, tf_est_masks
                
        def __call__(self):                                              
                # load all train data
                provider = DataProvider(data_folder_path=os.path.abspath('./../data/DSD100subset_DSampled/Sources'))
                bass_list, drums_list, other_list, vocals_list = provider.load_all_train_data()
                # split train valid
                train_bass_list,    valid_bass_list = provider.split_to_train_valid(bass_list)
                train_drums_list, valid_drums_list = provider.split_to_train_valid(drums_list)
                train_other_list,   valid_other_list = provider.split_to_train_valid(other_list)
                train_vocals_list,  valid_vocals_list = provider.split_to_train_valid(vocals_list)
                # set epoch num
                self.train_iter = int(len(train_bass_list) / self.batch_size)
                self.valid_iter = int(len(valid_bass_list) / self.batch_size)
                # define model
                tf_lr = tf.placeholder(tf.float32) # learning rate
                tf_mix = tf.placeholder(tf.float32, (None, 66304)) #Batch, Sample
                tf_target = tf.placeholder(tf.float32, (None, 66304)) #Batch,Sample
                
                tf_train_step, tf_loss , tf_est_spec, tf_target_spec, tf_mag_mix_spec, tf_ori_mix_spec, tf_est_masks = self.__model(tf_mix, tf_target, tf_lr)
                
                # GPU config
                config = tf.ConfigProto(
                        gpu_options=tf.GPUOptions(
                                visible_device_list=None, # specify GPU number
                                allow_growth = True
                        )
                )
                with tf.Session(config = config) as sess:
                        init = tf.global_variables_initializer()  
                        sess.run(init)
                        print("Start Training")
                        net_saver = NetSaver(saver_folder_name='MRUNet',  saver_file_name='mr_u_net')
                        early_stopping = EarlyStopping()
#                            random cutting, shffule, random scaling
                        train_data_argument = DataArgument(self.fs, self.sec, self.train_data_num)
                        train_arg_bass_array = train_data_argument(train_bass_list)
                        train_arg_drums_array = train_data_argument(train_drums_list)                                
                        train_arg_other_array = train_data_argument(train_other_list)
                        train_arg_vocals_array = train_data_argument(train_vocals_list)

                        valid_data_argument = DataArgument(self.fs, self.sec, self.valid_data_num)
                        valid_arg_bass_array = valid_data_argument(valid_bass_list)
                        valid_arg_drums_array = valid_data_argument(valid_drums_list)
                        valid_arg_other_array = valid_data_argument(valid_other_list)
                        valid_arg_vocals_array = valid_data_argument(valid_vocals_list)  
                        # mixing
                        train_mixed_array = AudioModule.mixing(
                                                                            train_arg_bass_array,
                                                                            train_arg_drums_array,
                                                                            train_arg_other_array,
                                                                            train_arg_vocals_array
                                                                    )
                        train_target_array = train_arg_vocals_array

                        valid_mixed_array = AudioModule.mixing(
                                                                            valid_arg_bass_array,
                                                                            valid_arg_drums_array,
                                                                            valid_arg_other_array,
                                                                            valid_arg_vocals_array
                                                                    )
                        valid_target_array = valid_arg_vocals_array
                        for epoch in range(self.epoch_num):
                                print('epoch:' + str(epoch))                                
                                tf.keras.backend.set_learning_phase(1)
                                for train_time in range(self.train_iter):
                                    sess.run(tf_train_step, feed_dict = {
                                           tf_mix: train_mixed_array[train_time*self.batch_size:(train_time+1)*self.batch_size, :self.sample_len],
                                           tf_target: train_target_array[train_time*self.batch_size:(train_time+1)*self.batch_size, :self.sample_len],
                                           tf_lr: self.lr_init
                                        }
                                     )
                                    
                                tmp_valid_loss_list = []       
                                tf.keras.backend.set_learning_phase(0)                               
                                for valid_time in range(self.valid_iter):
                                    valid_loss = sess.run(tf_loss, feed_dict = {
                                               tf_mix: valid_mixed_array[valid_time*self.batch_size:(valid_time+1)*self.batch_size, :self.sample_len],
                                               tf_target: valid_target_array[valid_time*self.batch_size:(valid_time+1)*self.batch_size, :self.sample_len],
                                               tf_lr:  0.
                                            }
                                         )
                                    tmp_valid_loss_list.append(valid_loss)
                                
                                self.valid_loss_list.append(np.mean(tmp_valid_loss_list))
                                
                                vmin = -70
                                vmax = 0
                                est_spec, target_spec, mag_mix_spec, ori_spec_mix, est_mask = sess.run([tf_est_spec, tf_target_spec, tf_mag_mix_spec , tf_ori_mix_spec, tf_est_masks], feed_dict ={
                                    tf_mix: valid_mixed_array[0:3],
                                    tf_target: valid_target_array[0:3],
                                    tf_lr: 0.
                                })
                    
                                est_spec = np.squeeze(est_spec, axis=-1)
                                target_spec = np.squeeze(target_spec, axis=-1)
                                mag_mix_spec = np.squeeze(mag_mix_spec, axis=-1)
                                est_mask = np.squeeze(est_mask, axis=-1)
                                print("original spec mix")
                                visualize_spec.plot_spec(ori_spec_mix[0], self.fs, self.sec, vmax, vmin)
                                print("magnitude spec mix")
                                visualize_spec.plot_log_spec(mag_mix_spec[0], self.fs, self.sec, 10, -10)
                                print(np.max(mag_mix_spec[0]))
                                print(np.min(mag_mix_spec[0]))
                                print("target spec")
                                visualize_spec.plot_spec(target_spec[0], self.fs, self.sec, vmax, vmin)
                                print("estimate spec")
                                visualize_spec.plot_spec(est_spec[0], self.fs, self.sec,  vmax, vmin)
                                print("est mask")
                                visualize_spec.plot_spec(est_mask[0], self.fs, self.sec, 1, 0)

                                visualize_loss.plot_loss(self.valid_loss_list)
                            
#                                 if epoch%9 ==  0:
#                                     net_saver(sess, step=epoch)
                        

if __name__ == '__main__':
    train = Train()
    train()



/Users/nagatomotaketo/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(?, 66304)
(?, 256, 513)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
mr_h1 (?, 256, 256, 1)
mr_h2 (?, 128, 256, 16)
(?, 128, 256, 16)
(?, 64, 128, 32)
(?, 32, 64, 64)
(?, 16, 32, 128)
(?, 8, 16, 256)
(?, 4, 8, 512)
(?, 8, 16, 256)
(?, 16, 32, 128)
(?, 32, 64, 64)
(?, 64, 128, 32)
(?, 128, 256, 16)
(?, 256, 512, 1)

Start Training


AttributeError: module 'model.DataArgument' has no attribute 'random_cutting'